# Cell Segmentation and Tracking

In [1]:
import numpy as np
from skimage import io as sio
from skimage import filters as sif
from skimage import measure as sms
from skimage import draw as sdr
from skimage import color as scl
from skimage import exposure as sxp
from skimage import morphology as smp
from IPython.display import HTML
%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib import animation as anim
from matplotlib import cm
plt.rcParams['figure.figsize'] = [12, 9]
plt.rcParams['image.cmap'] = 'gray'

# Filepath
filepath = 'data/Fluo-N2DL-HeLa'
collection_pattern = f'{filepath}/01/t*.tif'

# Utility functions
def frames(images):
    for i in range(images.shape[0]):
        yield i, images[i,:,:]

def show_images(images):
    fig = plt.figure()
    plt.xticks([])
    plt.yticks([])
    ims = []
    for frame, image in frames(images):
        im = plt.imshow(image, animated=True)
        ims.append([im])
    ani = anim.ArtistAnimation(fig, ims, interval=60, blit=True)
    plt.close(fig)
    return HTML(ani.to_html5_video())

## Segmentation

In [2]:
# Load an example image (and rescale it)
images = sio.imread_collection(collection_pattern, plugin='tifffile')
images = sio.concatenate_images(images)
for frame, image in frames(images):
    images[frame,:,:] = sxp.rescale_intensity(image, out_range='uint8')
show_images(images)

In [3]:
# Get image thresholds using Otsu's Method
thresh = []
for frame, image in frames(images):
    thresh.append(sif.threshold_otsu(image))
thresh = np.array(thresh)

# Apply threshold to images
thresh = thresh.reshape((thresh.shape[0], 1, 1))
binary_images = thresh <= images

# Show images
show_images(binary_images)

In [4]:
# Correct images using opening and closing
selem = smp.disk(5)
corrected_images = []
for frame, image in frames(binary_images):
    corrected = smp.binary_opening(image, selem)
    corrected = smp.binary_closing(corrected)
    corrected_images.append(corrected)
corrected_images = np.array(corrected_images)
show_images(corrected_images)